## GPT for style completion

In [1]:
from transformers import GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, GPT2LMHeadModel, pipeline, \
                         Trainer, TrainingArguments


/home/lucapug/.local/share/virtualenvs/oreilly-gpt-hands-on-nlg-eLFJLio5/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')  # load up a standard gpt2 model

tokenizer.pad_token = tokenizer.eos_token  
# set our pad token to be the eos token. This lets gpt know how to fill space


In [3]:
# load up our data into a dataset
pds_data = TextDataset(
    tokenizer=tokenizer,
    file_path='../data/PDS2.txt',  # Principles of Data Science - Sinan Ozdemir
    block_size=64  # length of each chunk of text to use as a datapoint
)

/home/lucapug/.local/share/virtualenvs/oreilly-gpt-hands-on-nlg-eLFJLio5/lib/python3.9/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [4]:
pds_data[0], pds_data[0].shape  # inspect the first point

(tensor([  198,   464,  1366,  3783,   569,  1697, 16362,   198,   464, 10688,
           198,   198, 16281,   784, 10922,   263,    12,  8344,  4872,  4981,
           198,   198, 34556,  8300,   198,  5195, 11361,    30,   198,   198,
         37906,  6593,   198, 16281,   286,  4096, 11361,   198, 16281,   784,
         32096,   257,  2060,  6126,   198,   198, 43961,  3725,   198,   198,
          4366,   517, 29191,   198,  6601,  3783,  1339,  3640,   198,   198,
         20448,  2050,   784,  3557]),
 torch.Size([64]))

In [5]:
print(tokenizer.decode(pds_data[0]))


The data science Venn diagram
The math

Example – spawner-recruit models

Computer programming
Why Python?

Python practices
Example of basic Python
Example – parsing a single tweet

Domain knowledge

Some more terminology
Data science case studies

Case study – autom


In [4]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,  
    # MLM is Masked Language Modelling (for BERT + auto-encoding tasks, while here we use gpt2 that is autoregressive (or causal) model)
)

In [7]:
# example of how collator pads data dynamically (our inputs to the model must be all of the same length)
collator_example = data_collator([tokenizer('I am an input'), tokenizer('So am I')])

collator_example

{'input_ids': tensor([[   40,   716,   281,  5128],
        [ 2396,   716,   314, 50256]]), 'attention_mask': tensor([[1, 1, 1, 1],
        [1, 1, 1, 0]]), 'labels': tensor([[  40,  716,  281, 5128],
        [2396,  716,  314, -100]])}

In [8]:
collator_example.input_ids  # 50256 is our pad token id

tensor([[   40,   716,   281,  5128],
        [ 2396,   716,   314, 50256]])

In [9]:
tokenizer.pad_token_id

50256

In [10]:
collator_example.attention_mask  # Note the 0 in the attention mask where we have a pad token

tensor([[1, 1, 1, 1],
        [1, 1, 1, 0]])

In [11]:
collator_example.labels  # note the -100 to ignore loss calculation for the padded token
# Labels are shifted inside the GPT model so we don't need to worry about that

tensor([[  40,  716,  281, 5128],
        [2396,  716,  314, -100]])

In [12]:
model = GPT2LMHeadModel.from_pretrained('gpt2')  # load up a GPT2 model

pretrained_generator = pipeline(  # create a generator with built in params
    'text-generation', model=model, tokenizer='gpt2',
    config={'max_length': 200, 'do_sample': True, 'top_p': 0.9, 'temperature': 0.7, 'top_k': 10}
)

In [13]:
print('----------')
for generated_sequence in pretrained_generator('This dataset shows the relationship', num_return_sequences=3):
    print(generated_sequence['generated_text'])
    print('----------')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


----------
This dataset shows the relationship between self-rating quality and perceived performance over time, from the mid-1990s until the mid-2015 period (see figure 1). Self-rating quality was first documented after 2002 when self-rating quality rose substantially with
----------
This dataset shows the relationship over time between the time between one epoch and the next epoch (SEM: CCD, CCD RAS: DRS, SEM for each time). The number of consecutive epochs with significant positive correlations suggests a trend
----------
This dataset shows the relationship between IQ and the prevalence of psychopathology and attention deficit hyperactivity disorder (ADHD), according to a meta-analysis of 15 case reports. The case reports consist of more than 4,000 individuals and were conducted using structured
----------


In [14]:
training_args = TrainingArguments(
    output_dir="./gpt2_pds", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=32,  # batch size for evaluation
    logging_steps=10,
    load_best_model_at_end=True,
    evaluation_strategy='epoch',
    save_strategy='epoch'
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=pds_data.examples[:int(len(pds_data.examples)*.8)],
    eval_dataset=pds_data.examples[int(len(pds_data.examples)*.8):]
)

trainer.evaluate()

{'eval_loss': 4.50043249130249,
 'eval_runtime': 4.2972,
 'eval_samples_per_second': 108.21,
 'eval_steps_per_second': 3.491}

In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.305100,3.472245
2,3.047800,3.436059
3,2.901100,3.436345


TrainOutput(global_step=177, training_loss=3.1588072480454956, metrics={'train_runtime': 535.8446, 'train_samples_per_second': 10.408, 'train_steps_per_second': 0.33, 'total_flos': 182153207808000.0, 'train_loss': 3.1588072480454956, 'epoch': 3.0})

In [16]:
trainer.evaluate()  # loss decrease is slowing down so we are hitting our limit

{'eval_loss': 3.4360592365264893,
 'eval_runtime': 9.5936,
 'eval_samples_per_second': 48.47,
 'eval_steps_per_second': 1.564,
 'epoch': 3.0}

In [17]:
trainer.save_model()

In [5]:
loaded_model = GPT2LMHeadModel.from_pretrained('./gpt2_pds')

finetuned_generator = pipeline(
    'text-generation', model=loaded_model, tokenizer=tokenizer,
    config={'max_length': 200, 'do_sample': True, 'top_p': 0.9, 'temperature': 0.7, 'top_k': 10}
)

In [6]:
# examples are now sustainably about data
print('----------')
for generated_sequence in finetuned_generator('This dataset shows the relationship', num_return_sequences=3):
    print(generated_sequence['generated_text'])
    print('----------')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


----------
This dataset shows the relationship between BMI and a variety of other variables in the world.

The main goal of this dataset is to make it able to be used to make predictions about the entire population of individuals and the
individual structure of the population
----------
This dataset shows the relationship for people on the autism spectrum versus those on the general population. This is called the test set. In the first column, the tests we have for the test set are
given the name, age and gender. We can
----------
This dataset shows the relationship of each
data point to the following. The first three are given a matrix of each data point in the dataset; the fourth is called the logistic relation, which
is the relationship between two sets of data points and
----------


## GPT for code dictation

In [7]:
from transformers import GPT2Tokenizer, DataCollatorForLanguageModeling, TrainingArguments, Trainer, \
                         GPT2LMHeadModel, pipeline
from datasets import Dataset
import pandas as pd

In [8]:
data = pd.read_csv('../data/english_to_latex.csv')

print(data.shape)

data.head(2)

(50, 2)


,English,LaTeX
0,integral from a to b of x squared,"\int_{a}^{b} x^2 \,dx"
1,integral from negative 1 to 1 of x squared,"\int_{-1}^{1} x^2 \,dx"


In [9]:
data.head(10)

,English,LaTeX
0,integral from a to b of x squared,"\int_{a}^{b} x^2 \,dx"
1,integral from negative 1 to 1 of x squared,"\int_{-1}^{1} x^2 \,dx"
2,integral from negative 1 to infinity of x cubed,"\int_{-1}^{\inf} x^3 \,dx"
3,integral from 0 to infinity of x squared,"\int_{0}^{\inf} x^2 \,dx"
4,integral from 0 to infinity of y squared,"\int_{0}^{\inf} y^2 \,dy"
5,integral from 1 to 2 of x over 2,"\int_{1}^{2} \frac{x}{2} \,dx"
6,f of x equals x squared,f(x) = x^2
7,h of x equals x squared,h(x) = x^2
8,g of x equals x squared,g(x) = x^2
9,g of x equals x to the eighth power,g(x) = x^8


In [10]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

tokenizer.pad_token = tokenizer.eos_token

# Add our singular prompt
CONVERSION_PROMPT = 'LCT\n'  # LaTeX conversion task

CONVERSION_TOKEN = 'LaTeX:'


In [11]:
# This is our "training prompt" that we want GPT2 to recognize and learn
training_examples = f'{CONVERSION_PROMPT}English: ' + data['English'] + '\n' + CONVERSION_TOKEN + ' ' + data['LaTeX'].astype(str)

print(training_examples[0])


LCT
English: integral from a to b of x squared
LaTeX: \int_{a}^{b} x^2 \,dx


In [12]:
task_df = pd.DataFrame({'text': training_examples})

task_df.head(2)

,text
0,LCT\nEnglish: integral from a to b of x square...
1,LCT\nEnglish: integral from negative 1 to 1 of...


In [13]:
# adding the EOS token at the end so the model knows when to stop predicting

task_df['text'] = task_df['text'].map(lambda x: f'{x}{tokenizer.eos_token}')

In [14]:
latex_data = Dataset.from_pandas(task_df)  # turn a pandas DataFrame into a Dataset

def preprocess(examples):  
    # tokenize our text but don't pad because our collator will pad for us dynamically
    return tokenizer(examples['text'], truncation=True)

latex_data = latex_data.map(preprocess, batched=True)

latex_data = latex_data.train_test_split(train_size=.8)

Map: 100%|██████████| 50/50 [00:00<00:00, 2252.07 examples/s]


In [15]:
latex_data['train'][0]

{'text': 'LCT\nEnglish: g of x equals x to the eighth power\nLaTeX: g(x) = x^8<|endoftext|>',
 'input_ids': [43,
  4177,
  198,
  15823,
  25,
  308,
  286,
  2124,
  21767,
  2124,
  284,
  262,
  16974,
  1176,
  198,
  14772,
  49568,
  25,
  308,
  7,
  87,
  8,
  796,
  2124,
  61,
  23,
  50256],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [16]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [17]:
latex_gpt2 = GPT2LMHeadModel.from_pretrained('gpt2')

In [18]:
latex_data

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 40
    })
    test: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 10
    })
})

In [19]:
training_args = TrainingArguments(
    output_dir="./english_to_latex",
    overwrite_output_dir=True, # overwrite the content of the output directory
    num_train_epochs=5, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=20,  # batch size for evaluation
    load_best_model_at_end=True,
    logging_steps=5,
    log_level='info',
    evaluation_strategy='epoch',
    save_strategy='epoch'
)

trainer = Trainer(
    model=latex_gpt2,
    args=training_args,
    train_dataset=latex_data["train"],
    eval_dataset=latex_data["test"],
    data_collator=data_collator,
)

trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10
  Batch size = 20


{'eval_loss': 5.097783088684082,
 'eval_runtime': 2.5611,
 'eval_samples_per_second': 3.905,
 'eval_steps_per_second': 0.39}

In [20]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 40
  Num Epochs = 5
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 50
  Number of trainable parameters = 124,439,808


Epoch,Training Loss,Validation Loss
1,2.453700,1.842356
2,1.223900,1.097196
3,0.852700,1.014565
4,0.677900,0.958824
5,0.643100,0.938436


The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10
  Batch size = 20
Saving model checkpoint to ./english_to_latex/checkpoint-10
Configuration saved in ./english_to_latex/checkpoint-10/config.json
Configuration saved in ./english_to_latex/checkpoint-10/generation_config.json
Model weights saved in ./english_to_latex/checkpoint-10/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10
  Batch size = 20
Saving model checkpoint to ./english_to_latex/checkpoint-20
Configuration saved in ./english_to_latex/checkpoint-20/confi

TrainOutput(global_step=50, training_loss=1.415231738090515, metrics={'train_runtime': 33.1676, 'train_samples_per_second': 6.03, 'train_steps_per_second': 1.507, 'total_flos': 3427416576000.0, 'train_loss': 1.415231738090515, 'epoch': 5.0})

In [21]:
trainer.evaluate()  # best loss of 0.8818739 (mine is 0.8392...)

The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10
  Batch size = 20


{'eval_loss': 0.9384355545043945,
 'eval_runtime': 0.3464,
 'eval_samples_per_second': 28.868,
 'eval_steps_per_second': 2.887,
 'epoch': 5.0}

In [143]:
# Let's try fine-tuning it again but first let's have the model read a math book

In [22]:
# Linear Algebra book by Jim Hefferon written in LaTeX for free - https://joshua.smcvt.edu/linearalgebra

book_data = TextDataset(
    tokenizer=tokenizer,
    file_path='../data/latex_cheat_sheet.tex',  # train on a LaTeX cheat sheet they made
    block_size=128
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,  # MLM is Masked Language Modelling
)

latex_gpt2 = GPT2LMHeadModel.from_pretrained('gpt2')

training_args = TrainingArguments(
    output_dir="./math_book",
    overwrite_output_dir=True, # overwrite the content of the output directory
    num_train_epochs=2, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=32,  # batch size for evaluation
    load_best_model_at_end=True,
    logging_steps=1,
    eval_steps=1,
    evaluation_strategy='epoch',
    save_strategy='epoch'
)

trainer = Trainer(
    model=latex_gpt2,
    args=training_args,
    data_collator=data_collator,
    train_dataset=book_data.examples[:int(len(book_data.examples)*.8)],
    eval_dataset=book_data.examples[int(len(book_data.examples)*.8):]
)

/home/lucapug/.local/share/virtualenvs/oreilly-gpt-hands-on-nlg-eLFJLio5/lib/python3.9/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Loading features from cached file ../data/cached_lm_GPT2Tokenizer_128_latex_cheat_sheet.tex [took 0.002 s]
loading configuration file config.json from cache at /home/lucapug/.cache/huggingface/hub/models--gpt2/snapshots/11c5a3d5811f50298f278a704980280950aedb10/config.json
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_

In [23]:
trainer.evaluate()  # initial loss for the math book

***** Running Evaluation *****
  Num examples = 21
  Batch size = 32


{'eval_loss': 3.3151752948760986,
 'eval_runtime': 0.6525,
 'eval_samples_per_second': 32.182,
 'eval_steps_per_second': 1.532}

In [24]:
trainer.train()

***** Running training *****
  Num examples = 80
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 6
  Number of trainable parameters = 124,439,808


Epoch,Training Loss,Validation Loss
1,2.824000,2.801510
2,2.344900,2.690235


***** Running Evaluation *****
  Num examples = 21
  Batch size = 32
Saving model checkpoint to ./math_book/checkpoint-3
Configuration saved in ./math_book/checkpoint-3/config.json
Configuration saved in ./math_book/checkpoint-3/generation_config.json
Model weights saved in ./math_book/checkpoint-3/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 21
  Batch size = 32
Saving model checkpoint to ./math_book/checkpoint-6
Configuration saved in ./math_book/checkpoint-6/config.json
Configuration saved in ./math_book/checkpoint-6/generation_config.json
Model weights saved in ./math_book/checkpoint-6/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./math_book/checkpoint-6 (score: 2.690234661102295).


TrainOutput(global_step=6, training_loss=2.6450769503911338, metrics={'train_runtime': 190.5986, 'train_samples_per_second': 0.839, 'train_steps_per_second': 0.031, 'total_flos': 10451681280000.0, 'train_loss': 2.6450769503911338, 'epoch': 2.0})

In [25]:
trainer.save_model()  # 2 epochs led to a pretty good drop in loss

Saving model checkpoint to ./math_book
Configuration saved in ./math_book/config.json
Configuration saved in ./math_book/generation_config.json
Model weights saved in ./math_book/pytorch_model.bin


In [26]:
math_latex_gpt2 = GPT2LMHeadModel.from_pretrained('./math_book')  # load up our gpt pre-trained

training_args = TrainingArguments(
    output_dir="./math_english_to_latex",
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=5, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=20,  # batch size for evaluation
    load_best_model_at_end=True,
    logging_steps=5,
    log_level='info',
    evaluation_strategy='epoch',
    save_strategy='epoch'
)

trainer = Trainer(
    model=math_latex_gpt2,
    args=training_args,
    train_dataset=latex_data["train"],
    eval_dataset=latex_data["test"],
    data_collator=data_collator,
)

trainer.evaluate()  # loss is starting slightly lower than before

loading configuration file ./math_book/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.34.0",
  "use_cache": true,
  "vocab_size": 50257
}

loading weights f

{'eval_loss': 4.602993488311768,
 'eval_runtime': 1.5501,
 'eval_samples_per_second': 6.451,
 'eval_steps_per_second': 0.645}

In [27]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 40
  Num Epochs = 5
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 50
  Number of trainable parameters = 124,439,808


Epoch,Training Loss,Validation Loss
1,2.253800,1.651878
2,1.164100,1.054832
3,0.848400,0.992001
4,0.670700,0.935122
5,0.627400,0.916801


The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10
  Batch size = 20
Saving model checkpoint to ./math_english_to_latex/checkpoint-10
Configuration saved in ./math_english_to_latex/checkpoint-10/config.json
Configuration saved in ./math_english_to_latex/checkpoint-10/generation_config.json
Model weights saved in ./math_english_to_latex/checkpoint-10/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10
  Batch size = 20
Saving model checkpoint to ./math_english_to_latex/checkpoint-20
Configuration saved in ./math_englis

TrainOutput(global_step=50, training_loss=1.335031304359436, metrics={'train_runtime': 25.8949, 'train_samples_per_second': 7.724, 'train_steps_per_second': 1.931, 'total_flos': 3427416576000.0, 'train_loss': 1.335031304359436, 'epoch': 5.0})

In [28]:
trainer.evaluate()  # pre-training on the book for one epoch led to a minor drop in loss

The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10
  Batch size = 20


{'eval_loss': 0.9168012738227844,
 'eval_runtime': 0.3015,
 'eval_samples_per_second': 33.167,
 'eval_steps_per_second': 3.317,
 'epoch': 5.0}

In [29]:
trainer.save_model()  # save this model

Saving model checkpoint to ./math_english_to_latex
Configuration saved in ./math_english_to_latex/config.json
Configuration saved in ./math_english_to_latex/generation_config.json
Model weights saved in ./math_english_to_latex/pytorch_model.bin


In [30]:
loaded_model = GPT2LMHeadModel.from_pretrained('./math_english_to_latex')
latex_generator = pipeline('text-generation', model=loaded_model, tokenizer=tokenizer)

loading configuration file ./math_english_to_latex/config.json
Model config GPT2Config {
  "_name_or_path": "./math_book",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.34.0",
  "use_cache": true,
  "vocab_size": 50257
}

In [31]:
text_sample = 'g of x equals integral from 0 to 1 of x squared'
conversion_text_sample = f'{CONVERSION_PROMPT}English: {text_sample}\n{CONVERSION_TOKEN}'

print(conversion_text_sample)

LCT
English: g of x equals integral from 0 to 1 of x squared
LaTeX:


In [32]:
print(latex_generator(
    conversion_text_sample, num_beams=5, early_stopping=True, temperature=0.7,
    max_length=len(tokenizer.encode(conversion_text_sample)) + 20
)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


LCT
English: g of x equals integral from 0 to 1 of x squared
LaTeX: g(x) = \frac{x}{1} \,dx^2
LaTeX


In [33]:
# Another example
text_sample = 'r of x is sum from 0 to x of x squared'
conversion_text_sample = f'{CONVERSION_PROMPT}English: {text_sample}\n{CONVERSION_TOKEN}'

print(latex_generator(
    conversion_text_sample, num_beams=5, early_stopping=True, temperature=0.7,
    max_length=len(tokenizer.encode(conversion_text_sample)) + 20
)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


LCT
English: r of x is sum from 0 to x of x squared
LaTeX: r(x) = \frac{x}{x}^2
LaTeX: \frac


In [34]:
print(latex_generator(
    text_sample, num_beams=5, early_stopping=True, temperature=0.7,
    max_length=len(tokenizer.encode(conversion_text_sample)) + 20
)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


r of x is sum from 0 to x of x squared
LaTeX: \sum_{0}^{x} x^2 \,dx^2 \,dx^2 \,dx^


In [35]:
# Another example
text_sample = 'pi to the 8th power'
conversion_text_sample = f'{CONVERSION_PROMPT}English: {text_sample}\n{CONVERSION_TOKEN}'

print(latex_generator(
    conversion_text_sample, num_beams=5, early_stopping=True, temperature=.7,
    max_length=len(tokenizer.encode(conversion_text_sample)) + 20
)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


LCT
English: pi to the 8th power
LaTeX: \pi^8 \pi^8 \,dx
LaTeX: \pi^8 \


In [36]:
# Sanity check that a non-finetuned model could not have done this
non_finetuned_latex_generator = pipeline(
    'text-generation', 
    model=GPT2LMHeadModel.from_pretrained('gpt2'),  # not fine-tuned!
    tokenizer=tokenizer
)

loading configuration file config.json from cache at /home/lucapug/.cache/huggingface/hub/models--gpt2/snapshots/11c5a3d5811f50298f278a704980280950aedb10/config.json
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.34.0",
  "

In [37]:
few_shot_prompt = """LCT
English: f of x is sum from 0 to x of x squared
LaTeX: f(x) = \sum_{0}^{x} x^2 \,dx \
###
LCT
English: f of x equals integral from 0 to pi of x to the fourth power
LaTeX: f(x) = \int_{0}^{\pi} x^4 \,dx \
###
LCT
English: pi to the 8th power
LaTeX:"""

print(non_finetuned_latex_generator(
    few_shot_prompt, num_beams=5, early_stopping=True, temperature=0.7,
    max_length=len(tokenizer.encode(few_shot_prompt)) + 20
)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


LCT
English: f of x is sum from 0 to x of x squared
LaTeX: f(x) = \sum_{0}^{x} x^2 \,dx ###
LCT
English: f of x equals integral from 0 to pi of x to the fourth power
LaTeX: f(x) = \int_{0}^{\pi} x^4 \,dx ###
LCT
English: pi to the 8th power
LaTeX: pi to the 16th power
LaTeX: pi to the 32nd power
LaTeX:


In [38]:
# try another prompt
few_shot_prompt = """English to LaTeX
English: f of x is sum from 0 to x of x squared
LaTeX: f(x) = \sum_{0}^{x} x^2 \,dx \
###
LCT
English: f of x equals integral from 0 to pi of x to the fourth power
LaTeX: f(x) = \int_{0}^{\pi} x^4 \,dx \
###
LCT
English: x to the eighth power
LaTeX:"""

print(non_finetuned_latex_generator(
    few_shot_prompt, num_beams=5, early_stopping=True, temperature=0.7,
    max_length=len(tokenizer.encode(few_shot_prompt)) + 20
)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


English to LaTeX
English: f of x is sum from 0 to x of x squared
LaTeX: f(x) = \sum_{0}^{x} x^2 \,dx ###
LCT
English: f of x equals integral from 0 to pi of x to the fourth power
LaTeX: f(x) = \int_{0}^{\pi} x^4 \,dx ###
LCT
English: x to the eighth power
LaTeX: f(x) = \int_{0}^{\pi} x^4 \,dx


In [39]:
print(non_finetuned_latex_generator(
    conversion_text_sample, num_beams=5, early_stopping=True, temperature=0.7,
    max_length=len(tokenizer.encode(conversion_text_sample)) + 20
)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


LCT
English: pi to the 8th power
LaTeX: pi to the 8th power

LaTeX: pi to the 8th power

La
